
*   Preproc
1. Mounting drive
2. Git clone casa dir 
3. restart runtime after installing new spaCy
*   Proc
1. Get segs
2. feed segs into SpaCy model   
3. Pickle the token.infos needed

In [ ]:
# colab default spaCy needs to be upgraded before using zh_core_web_sm
!pip install --upgrade spacy
!python -m spacy download zh_core_web_sm
### Restart runtime after downloading else loading model will throw error

In [1]:
# https://drive.google.com/drive/u/0/folders/1hJQy62ZarJnrF5UTzR_tKN452my-8n2t
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import getpass, os
username = getpass.getpass("Github username: ")
password = getpass.getpass("Github password: ")
os.environ["CRED"] = f"{username}:{password}"
!git clone https://${CRED}@github.com/lopentu/casa
del username, password
!pip install -r /content/casa/requirements.txt

In [2]:
%load_ext autoreload
%autoreload 2
import pickle 
import sys
sys.path = ["/content/casa/src"] + sys.path
import casa

In [3]:
###!! Change the path if needed
#path = '/content/drive/MyDrive/指向情緒案/data/raw-data-cht-202010.pkl'
path = '/content/drive/MyDrive/指向情緒案/data/eda/ckip/raw-data-cht-202010.ckip.pkl'
with open (path, 'rb') as File:
  seg_threads = pickle.load(File)
print(type(seg_threads))
print(type(seg_threads[0]))

<class 'list'>
<class 'casa.opinion_thread.OpinionThread'>


In [4]:
import copy 
def make_tokens(temp):   
    ''''input: list of seg tokens; return a list of tokens in one reply/title'''
    # token.head.i: the index of the token in temp (token list), starting from 0
    # print(token, token.pos_, token.dep_, token.head, token.head.i)
    temp2 = ' '.join(temp)
    doc = nlp({"raw": ''.join(temp2), "words": temp})
    res = []
    for token in doc:
        tok = token.text
        pos = str(token.pos_)
        dep = str(token.dep_)
        head = str(token.head)
        head_ind = int(str(token.head.i))
        res.append((tok, pos, dep, head, head_ind))
    return res 

In [8]:
import spacy
from spacy.lang.zh import Chinese
from casa.custom_zh_defaults import CustomChineseDefaults
Chinese.Defaults = CustomChineseDefaults
nlp = spacy.load('zh_core_web_sm')
# 斷詞後
# https://stackoverflow.com/questions/50742516/how-to-get-the-index-of-a-token-in-a-sentence-in-spacy
import time
start_time = time.time()
for thread in seg_threads: # len: 7772
  if thread.main != None:
    # doc = nlp({"raw": "".join(words), "words": words.split()})
    temp = thread.main.text_tokens
    if temp != None:
      feed_to_title_tree = make_tokens(temp)
      thread.main.title_tree = feed_to_title_tree
      # .title_tree = [{token1, token1.dep_, token1.head, token1.head.i}, {token2, ...}]
    else:
      thread.main.title_tree = None
  if thread.replies != None:
      seg_replies = thread.replies
      for i, reply in enumerate(seg_replies):
        temp = reply.text_tokens # if temp is not []
        if temp != None:
          feed_to_text_tree = make_tokens(temp)
        #print(make_tokens(temp))
          thread.replies[i].text_tree = feed_to_text_tree
        else:
          thread.replies[i].text_tree = None
print("--- %s seconds ---" % (time.time() - start_time))
    #for reply in thread.replies:

nlp <spacy.lang.zh.Chinese object at 0x7f0715339550>
--- 400.2727336883545 seconds ---


In [10]:
tree = seg_threads[9].replies[-1].text_tree
print(type(tree[1][1]))
print(tree)

<class 'str'>
[('從', 'VERB', 'amod', 'hami video', 1), ('hami video', 'PROPN', 'nsubj', '進去', 2), ('進去', 'NOUN', 'ROOT', '進去', 2), ('Fox+', 'PROPN', 'compound:nn', '無法', 5), ('館', 'ADV', 'amod', '無法', 5), ('無法', 'NOUN', 'dobj', '進去', 2), ('看', 'VERB', 'conj', '進去', 2), ('直播', 'NOUN', 'dobj', '看', 6), ('，', 'PUNCT', 'punct', '進去', 2), ('要不要', 'ADV', 'nsubj', '趕快', 10), ('趕快', 'VERB', 'conj', '進去', 2), ('檢查', 'PROPN', 'dobj', '趕快', 10), ('哪', 'PRON', 'advmod', '對', 16), ('邊', 'VERB', 'mark:clf', '哪', 12), ('設定', 'VERB', 'dep', '哪', 12), ('不', 'ADV', 'neg', '對', 16), ('對', 'VERB', 'conj', '趕快', 10), ('？', 'PUNCT', 'punct', '進去', 2), ('看', 'VERB', 'dep', '～', 26), ('不', 'ADV', 'dep', '看', 18), ('到', 'VERB', 'dep', '看', 18), ('好', 'VERB', 'amod', '著急', 22), ('著急', 'NOUN', 'dobj', '看', 18), ('啊', 'PART', 'discourse', '看', 18), ('啊', 'PART', 'dep', '看', 18), ('啊', 'INTJ', 'discourse', '～', 26), ('～', 'PUNCT', 'ROOT', '～', 26)]


In [ ]:
# (怎麼說, 'VERB', 'advmod', B1 , 0)
# head 是 B1

##Saving outputs with pickle

In [13]:
import pickle

!mkdir -p /content/drive/MyDrive/指向情緒案/data/eda/spaCy_pipeline # -p check if this dir not already exist 
filename = 'raw-data-cht-2020-ppline.pkl'
with open(f"/content/drive/MyDrive/指向情緒案/data/eda/spaCy_pipeline/{filename}", "wb") as fout:
    pickle.dump(seg_threads, fout)

In [15]:
# test pickling
with open(f"/content/drive/MyDrive/指向情緒案/data/eda/spaCy_pipeline/{filename}", "rb") as fin:
  new_threads = pickle.load(fin)
print(new_threads[0].main.title_tree)
print(new_threads[0].replies[2].text_tree)

[('最近', 'NOUN', 'nmod:tmod', '打電話', 5), ('各', 'DET', 'det', '電信', 3), ('家', 'NOUN', 'mark:clf', '各', 1), ('電信', 'ADV', 'nsubj', '打電話', 5), ('一直', 'ADV', 'advmod', '打電話', 5), ('打電話', 'VERB', 'nsubj', '傳', 6), ('傳', 'NOUN', 'ROOT', '傳', 6), ('簡訊', 'VERB', 'compound:nn', '詢問', 8), ('詢問', 'NOUN', 'dep', '續約', 10), ('要不要', 'NOUN', 'nsubj', '續約', 10), ('續約', 'VERB', 'ROOT', '續約', 10), ('  ', 'SPACE', '', '續約', 10), ('有夠', 'ADV', 'advmod', '煩', 13), ('煩', 'VERB', 'dep', '合約', 17), ('的', 'PART', 'mark', '煩', 13), ('但', 'ADV', 'advmod', '合約', 17), ('畢竟', 'VERB', 'advmod', '合約', 17), ('合約', 'NOUN', 'dep', '有', 18), ('有', 'VERB', 'ROOT', '有', 18), ('30', 'NUM', 'nummod', '月', 21), ('個', 'NOUN', 'mark:clf', '30', 19), ('月', 'NOUN', 'compound:nn', ' 而', 22), (' 而', 'PROPN', 'dep', '普及', 26), ('5G', 'PUNCT', 'punct', '普及', 26), ('未來', 'VERB', 'nmod:tmod', '普及', 26), ('會', 'NOUN', 'advmod', '普及', 26), ('普及', 'VERB', 'ccomp', '有', 18), ('也', 'ADV', 'advmod', '不變', 29), ('是', 'VERB', 'cop', '不變', 29), 